In [1]:
import os
path = os.getcwd()
print(path)

/Users/akshitgupta/Downloads/MMSR/BPR-MF :FPMC


In [2]:
import sys
import numpy as np
import pandas as pd
from math import ceil
from itertools import islice
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix, dok_matrix
from tqdm import trange

In [3]:
file_dir = 'datasets/ml-1m'
file_path = os.path.join(path, file_dir,'ratings.dat')
print(file_path)

/Users/akshitgupta/Downloads/MMSR/BPR-MF :FPMC/datasets/ml-1m/ratings.dat


In [4]:
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv(file_path, sep = '::', names = names)
print('data dimension: \n', df.shape)
df.head()

<ipython-input-4-62dd7f865362>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_path, sep = '::', names = names)


data dimension: 
 (1000209, 4)


,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
def create_matrix(data, users_col, items_col, ratings_col, threshold = None):
   
    if threshold is not None:
        data = data[data[ratings_col] >= threshold]
        data[ratings_col] = 1
    
    for col in (items_col, users_col, ratings_col):
        print(col)
        data[col] = data[col].astype('category')

    ratings = csr_matrix((data[ratings_col],
                          (data[users_col].cat.codes, data[items_col].cat.codes)))
    ratings.eliminate_zeros()
    return ratings, data

In [6]:
items_col = 'item_id'
users_col = 'user_id'
ratings_col = 'rating'
# rating threshold, ratings above 3 considered as 1, below considered 0
threshold = 1
X, df2 = create_matrix(df, users_col, items_col, ratings_col, threshold)
X

item_id
user_id
rating


<6040x3706 sparse matrix of type '<class 'numpy.longlong'>'
	with 1000209 stored elements in Compressed Sparse Row format>

In [7]:
def create_train_test(ratings, test_size = 0.2, seed = 1234):
    assert test_size < 1.0 and test_size > 0.0

    # Dictionary Of Keys based sparse matrix is more efficient
    # for constructing sparse matrices incrementally compared with csr_matrix
    train = ratings.copy().todok()
    test = dok_matrix(train.shape)
    
    # for all the users assign randomly chosen interactions
    # to the test and assign those interactions to zero in the training;
    # when computing the interactions to go into the test set, 
    # remember to round up the numbers (e.g. a user has 4 ratings, if the
    # test_size is 0.2, then 0.8 ratings will go to test, thus we need to
    # round up to ensure the test set gets at least 1 rating)
    rstate = np.random.RandomState(seed)
    for u in range(ratings.shape[0]):
        split_index = ratings[u].indices
        n_splits = ceil(test_size * split_index.shape[0])
        test_index = rstate.choice(split_index, size = n_splits, replace = False)
        test[u, test_index] = ratings[u, test_index]
        train[u, test_index] = 0
    
    train, test = train.tocsr(), test.tocsr()
    return train, test

In [8]:
X_train, X_test = create_train_test(X, test_size = 0.2, seed = 1234)
print(X_train.shape)
print(X_test.shape)

(6040, 3706)
(6040, 3706)


In [9]:
class BPR:
    """
    Bayesian Personalized Ranking (BPR) for implicit feedback data

    Parameters
    ----------
    learning_rate : float, default 0.01
        learning rate for gradient descent

    n_factors : int, default 20
        Number/dimension of user and item latent factors

    n_iters : int, default 15
        Number of iterations to train the algorithm
        
    batch_size : int, default 1000
        batch size for batch gradient descent, the original paper
        uses stochastic gradient descent (i.e., batch size of 1),
        but this can make the training unstable (very sensitive to
        learning rate)

    reg : int, default 0.01
        Regularization term for the user and item latent factors

    seed : int, default 1234
        Seed for the randomly initialized user, item latent factors

    verbose : bool, default True
        Whether to print progress bar while training

    Attributes
    ----------
    user_factors : 2d ndarray, shape [n_users, n_factors]
        User latent factors learnt

    item_factors : 2d ndarray, shape [n_items, n_factors]
        Item latent factors learnt

    References
    ----------
    S. Rendle, C. Freudenthaler, Z. Gantner, L. Schmidt-Thieme 
    Bayesian Personalized Ranking from Implicit Feedback
    - https://arxiv.org/abs/1205.2618
    """
    def __init__(self, learning_rate = 0.01, n_factors = 15, n_iters = 10, 
                 batch_size = 1000, reg = 0.01, seed = 1234, verbose = True):
        self.reg = reg
        self.seed = seed
        self.verbose = verbose
        self.n_iters = n_iters
        self.n_factors = n_factors
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        
        # to avoid re-computation at predict
        self._prediction = None
        
    def fit(self, ratings):
        """
        Parameters
        ----------
        ratings : scipy sparse csr_matrix, shape [n_users, n_items]
            sparse matrix of user-item interactions
        """
        indptr = ratings.indptr
        indices = ratings.indices
        n_users, n_items = ratings.shape
        
        # ensure batch size makes sense, since the algorithm involves
        # for each step randomly sample a user, thus the batch size
        # should be smaller than the total number of users or else
        # we would be sampling the user with replacement
        batch_size = self.batch_size
        if n_users < batch_size:
            batch_size = n_users
            sys.stderr.write('WARNING: Batch size is greater than number of users,'
                             'switching to a batch size of {}\n'.format(n_users))

        batch_iters = n_users // batch_size
        
        # initialize random weights
        rstate = np.random.RandomState(self.seed)
        self.user_factors = rstate.normal(size = (n_users, self.n_factors))
        self.item_factors = rstate.normal(size = (n_items, self.n_factors))
        
        # progress bar for training iteration if verbose is turned on
        loop = range(self.n_iters)
        if self.verbose:
            loop = trange(self.n_iters, desc = self.__class__.__name__)
        
        for _ in loop:
            for _ in range(batch_iters):
                sampled = self._sample(n_users, n_items, indices, indptr)
                sampled_users, sampled_pos_items, sampled_neg_items = sampled
                self._update(sampled_users, sampled_pos_items, sampled_neg_items)

        return self
    
    def _sample(self, n_users, n_items, indices, indptr):
        """sample batches of random triplets u, i, j"""
        sampled_pos_items = np.zeros(self.batch_size, dtype = np.int)
        sampled_neg_items = np.zeros(self.batch_size, dtype = np.int)
        sampled_users = np.random.choice(
            n_users, size = self.batch_size, replace = False)
       
        for idx, user in enumerate(sampled_users):
            pos_items = indices[indptr[user]:indptr[user + 1]]
            
            if(len(pos_items)>0):
                pos_item = np.random.choice(pos_items)
                sampled_pos_items[idx] = pos_item
            neg_item = np.random.choice(n_items)
            while neg_item in pos_items:
                neg_item = np.random.choice(n_items)

            
            sampled_neg_items[idx] = neg_item

        return sampled_users, sampled_pos_items, sampled_neg_items
                
    def _update(self, u, i, j):
        """
        update according to the bootstrapped user u, 
        positive item i and negative item j
        """
        user_u = self.user_factors[u]
        item_i = self.item_factors[i]
        item_j = self.item_factors[j]
        
        # decompose the estimator, compute the difference between
        # the score of the positive items and negative items; a
        # naive implementation might look like the following:
        # r_ui = np.diag(user_u.dot(item_i.T))
        # r_uj = np.diag(user_u.dot(item_j.T))
        # r_uij = r_ui - r_uj
        
        # however, we can do better, so
        # for batch dot product, instead of doing the dot product
        # then only extract the diagonal element (which is the value
        # of that current batch), we perform a hadamard product, 
        # i.e. matrix element-wise product then do a sum along the column will
        # be more efficient since it's less operations
        # http://people.revoledu.com/kardi/tutorial/LinearAlgebra/HadamardProduct.html
        # r_ui = np.sum(user_u * item_i, axis = 1)
        #
        # then we can achieve another speedup by doing the difference
        # on the positive and negative item up front instead of computing
        # r_ui and r_uj separately, these two idea will speed up the operations
        # from 1:14 down to 0.36
        r_uij = np.sum(user_u * (item_i - item_j), axis = 1)
        sigmoid = np.exp(-r_uij) / (1.0 + np.exp(-r_uij))
        
        # repeat the 1 dimension sigmoid n_factors times so
        # the dimension will match when doing the update
        sigmoid_tiled = np.tile(sigmoid, (self.n_factors, 1)).T

        # update using gradient descent
        grad_u = sigmoid_tiled * (item_j - item_i) + self.reg * user_u
        grad_i = sigmoid_tiled * -user_u + self.reg * item_i
        grad_j = sigmoid_tiled * user_u + self.reg * item_j
        self.user_factors[u] -= self.learning_rate * grad_u
        self.item_factors[i] -= self.learning_rate * grad_i
        self.item_factors[j] -= self.learning_rate * grad_j
        return self

    def predict(self):
        """
        Obtain the predicted ratings for every users and items
        by doing a dot product of the learnt user and item vectors.
        The result will be cached to avoid re-computing it every time
        we call predict, thus there will only be an overhead the first
        time we call it. Note, ideally you probably don't need to compute
        this as it returns a dense matrix and may take up huge amounts of
        memory for large datasets
        """
        if self._prediction is None:
            self._prediction = self.user_factors.dot(self.item_factors.T)

        return self._prediction

    def _predict_user(self, user):
        """
        returns the predicted ratings for the specified user,
        this is mainly used in computing evaluation metric
        """
        user_pred = self.user_factors[user].dot(self.item_factors.T)
        return user_pred

    def recommend(self, ratings, N = 5):
        """
        Returns the top N ranked items for given user id,
        excluding the ones that the user already liked
        
        Parameters
        ----------
        ratings : scipy sparse csr_matrix, shape [n_users, n_items]
            sparse matrix of user-item interactions 
        
        N : int, default 5
            top-N similar items' N
        
        Returns
        -------
        recommendation : 2d ndarray, shape [number of users, N]
            each row is the top-N ranked item for each query user
        """
        n_users = ratings.shape[0]
        recommendation = np.zeros((n_users, N), dtype = np.uint32)
        for user in range(n_users):
            top_n = self._recommend_user(ratings, user, N)
            recommendation[user] = top_n

        return recommendation

    def _recommend_user(self, ratings, user, N):
        """the top-N ranked items for a given user"""
        scores = self._predict_user(user)
        print("scores")
        print(scores)
        # compute the top N items, removing the items that the user already liked
        # from the result and ensure that we don't get out of bounds error when 
        # we ask for more recommendations than that are available
        liked = set(ratings[user].indices)
        count = N + len(liked)
        if count < scores.shape[0]:

            # when trying to obtain the top-N indices from the score,
            # using argpartition to retrieve the top-N indices in 
            # unsorted order and then sort them will be faster than doing
            # straight up argort on the entire score
            # http://stackoverflow.com/questions/42184499/cannot-understand-numpy-argpartition-output
            ids = np.argpartition(scores, -count)[-count:]
            best_ids = np.argsort(scores[ids])[::-1]
            best = ids[best_ids]
        else:
            best = np.argsort(scores)[::-1]

        top_n = list(islice((rec for rec in best if rec not in liked), N))
        return top_n
    
    def get_similar_items(self, N = 5, item_ids = None):
        """
        return the top N similar items for itemid, where
        cosine distance is used as the distance metric
        
        Parameters
        ----------
        N : int, default 5
            top-N similar items' N
            
        item_ids : 1d iterator, e.g. list or numpy array, default None
            the item ids that we wish to find the similar items
            of, the default None will compute the similar items
            for all the items
        
        Returns
        -------
        similar_items : 2d ndarray, shape [number of query item_ids, N]
            each row is the top-N most similar item id for each
            query item id
        """
        # cosine distance is proportional to normalized euclidean distance,
        # thus we normalize the item vectors and use euclidean metric so
        # we can use the more efficient kd-tree for nearest neighbor search;
        # also the item will always to nearest to itself, so we add 1 to 
        # get an additional nearest item and remove itself at the end
        normed_factors = normalize(self.item_factors)
        knn = NearestNeighbors(n_neighbors = N + 1, metric = 'euclidean')
        knn.fit(normed_factors)

        # returns a distance, index tuple,
        # we don't actually need the distance
        if item_ids is not None:
            normed_factors = normed_factors[item_ids]

        _, items = knn.kneighbors(normed_factors)
        similar_items = items[:, 1:].astype(np.uint32)
        return similar_items

In [10]:

bpr_params = {'reg': 0.01,
              'learning_rate': 0.1,
              'n_iters': 160,
              'n_factors': 15,
              'batch_size': 100}

bpr = BPR(**bpr_params)
bpr.fit(X_train)

BPR: 100%|██████████| 160/160 [00:37<00:00,  4.27it/s]


In [11]:
def auc_score(model, ratings):
    """
    computes area under the ROC curve (AUC).
    The full name should probably be mean
    auc score as it is computing the auc
    for every user's prediction and actual
    interaction and taking the average for
    all users
    
    Parameters
    ----------
    model : BPR instance
        Trained BPR model
        
    ratings : scipy sparse csr_matrix, shape [n_users, n_items]
        sparse matrix of user-item interactions
    
    Returns
    -------
    auc : float 0.0 ~ 1.0
    """
    auc = 0.0
    n_users, n_items = ratings.shape
    for user, row in enumerate(ratings):
        y_pred = model._predict_user(user)
        
        y_true = np.zeros(n_items)
        print(y_pred)
        y_true[row.indices] = 1
        if(np.unique(y_true).size>1):
            auc += roc_auc_score(y_true, y_pred)

    auc /= n_users
    return auc

In [12]:
print(auc_score(bpr, X_train))
print(auc_score(bpr, X_test))

[ 3.92037176  3.27680457  1.26365154 ... -0.55006471 -3.05304743
  3.46472965]
[ 3.56094197  1.17763847  1.93899515 ... -1.59849528 -2.69070004
  1.95123894]
[ 3.31529125  1.21372552  1.70971963 ... -0.44382261 -1.30192284
  2.08701166]
[ 3.62201974  2.26395876  2.50396302 ... -1.72524495 -3.6536141
  1.99534518]
[ 2.37084626 -0.14559474  0.4366446  ... -0.60603417 -0.27899496
  0.95526757]
[ 2.5545919   2.51186329  1.34747792 ... -2.11453457 -0.89110331
  0.93001346]
[ 3.7794387   2.6478506   2.28642557 ... -2.03509083 -3.6479362
  3.05281082]
[ 2.39842854  1.10360123  2.02969607 ... -0.34077662 -2.16406893
  0.4882391 ]
[ 4.15367015  1.27425784  2.17514011 ... -0.72753535 -2.40752157
  1.68326371]
[ 2.97603661  2.48456889  0.92867898 ... -1.50735769 -1.98603572
  2.81936489]
[ 2.81145929  0.16173266  1.71947563 ... -2.0191673  -1.9022126
  2.19619489]
[ 3.59986158  1.40304015 -0.13115077 ... -2.21038776 -0.71098696
  1.5107001 ]
[ 3.47232092  2.24029774  2.26250346 ... -1.56522712 -3

[ 4.20506532  0.20674513  1.29340462 ... -1.92621964 -2.73241169
  1.78329136]
[ 4.94422646  1.45999838  0.90480667 ... -0.29396161 -1.0361914
  1.64432527]
[ 2.04932695  1.08214477  1.78221269 ... -0.90972873 -2.92143975
  1.7183633 ]
[ 3.03195005  1.65335024  1.68440294 ... -0.31197961 -2.99009026
  0.66950933]
[ 1.38887875  1.82231191  1.02597943 ... -2.61608198 -1.20386573
  1.42776884]
[ 1.80944286  1.47505981  1.73791974 ... -1.09162328 -2.83795937
  1.77519703]
[ 1.42882982  0.94516734  1.33538993 ... -1.79423074 -2.32879889
  0.85125287]
[ 3.47074505  1.87862334  2.35608159 ...  0.08993173 -2.03527764
  2.8704723 ]
[ 3.2316412   1.50725358  0.90705026 ... -0.67320291 -1.12153651
  1.48321476]
[ 3.39793163  0.54107359  0.71499826 ... -4.06393447 -3.6940692
  1.95005807]
[ 2.49570905  2.43456162  1.86699223 ... -1.45255237 -3.53852117
  1.42615794]
[ 4.26400189  1.6218876   1.43765711 ... -1.9611634  -3.45766432
  3.56908406]
[ 2.93791818  0.52556916  1.18639754 ... -1.85460058 -

[ 4.20040097  1.44994006  2.24812317 ... -0.01732325 -1.12255956
  1.43068285]
[ 2.45305088  1.17695726  0.84793285 ... -1.50899195 -1.76281584
  2.90710057]
[ 2.15087441  0.62841325  0.97428723 ... -1.67332458 -1.00340915
  0.99369462]
[ 3.30573051  1.14163671  2.11461809 ...  0.82678771 -0.53917512
  3.38921653]
[ 2.54660602  1.73362182  2.98690957 ...  0.89354125 -2.85294555
  3.1638188 ]
[ 2.45315319  0.06793497  2.09611058 ... -0.7425382  -2.93566213
  1.30334178]
[ 2.78182131  0.67996319  1.23059893 ... -2.01126188 -2.11838783
  2.29409474]
[ 3.00612726  1.19995931  2.97217352 ... -0.84319145 -3.24153199
  0.91542716]
[ 3.58974979  2.15766586  1.40394588 ... -1.01494736 -1.93495847
  2.00128458]
[ 3.20719558  1.30928917  1.0373604  ... -0.90755189 -1.4605674
  1.29013417]
[ 3.42399765  0.43935755  1.3227469  ... -1.512284   -2.95456448
  2.61695092]
[ 3.23887054  1.43283204  0.77088339 ... -0.70052512 -1.00986434
  0.33308059]
[ 3.88185655  2.33759036  1.90982269 ... -1.81944772 

[ 3.04965645  0.90556177  1.24089028 ... -1.16590801 -0.62308085
  1.59248739]
[ 3.93070669  2.70651841  1.27691275 ... -1.24055664 -1.79096962
  1.94963059]
[ 2.93032881  0.68618177  1.40316612 ... -1.58718186 -2.55918357
  2.25701374]
[ 3.75096625  1.75435386  2.16888808 ...  0.13943946 -2.23700092
  3.54276757]
[ 2.33755223e+00  6.50595083e-01 -4.51019966e-04 ... -1.45546751e+00
 -1.57717193e+00  7.02631695e-01]
[ 2.24271771  0.43439166  1.89750353 ... -0.02559738 -0.85089892
  0.24310201]
[ 1.89281269  1.43466177  1.44083251 ... -1.00557658 -1.01099781
  1.00275663]
[ 1.6735664   1.85628783  1.20248355 ... -1.54922327 -0.58525024
  1.67432331]
[ 2.65502973  1.93094601  0.64058361 ... -2.33100544 -2.51437051
  2.43464714]
[ 3.60581846  2.09863389  2.58091542 ... -2.18644777 -3.30725023
  2.20484896]
[ 1.13202512 -0.06859778  0.48717153 ... -1.4544168   0.35134375
  0.16240942]
[ 5.59131527  0.71742461  0.74429621 ... -3.70602724 -2.39416543
  0.59131855]
[ 4.03045803  2.00103874  0.

[ 4.08739418  1.3707885   0.88784023 ... -0.5150665  -2.15033747
  1.89972215]
[ 4.20598051  1.72858749  1.10807933 ... -0.7223956  -2.33317908
  2.66711864]
[ 3.23439635  1.14748637  1.84091304 ... -2.40934249 -1.65287706
  1.81167082]
[ 3.66868568  0.95412421  1.86950381 ... -0.35913857 -1.35328551
  0.8179563 ]
[ 3.39675373  2.01113023  1.73263366 ...  0.49200442 -3.41084811
  2.4081232 ]
[ 2.7268789   2.50119335  0.95362556 ... -3.17057082 -2.24186538
  0.7493885 ]
[ 1.30157147  0.28773154  0.74999699 ... -1.04735936 -1.66918599
  0.45043662]
[ 4.17397941  1.86148291  0.01682656 ... -3.19139602 -1.67707132
  1.5478616 ]
[ 4.78340302  0.85294725  1.87582558 ... -1.35354385 -3.99039402
  2.64425497]
[ 3.38843105  1.26226863  2.05808182 ... -1.06541338 -1.90153377
  2.23616152]
[ 3.43701929  0.08676808  1.12465955 ... -2.83660055 -2.69471122
  2.42760575]
[ 3.43509714  1.01080533  1.25368644 ... -1.66061545 -2.00521586
  1.04380096]
[ 2.68175482  1.51253646  2.63089884 ... -1.63583754

...

[ 2.93284596  0.93717137  3.09618193 ... -0.03123878 -2.19673916
  1.53631083]
[ 3.58622467  0.58233477  1.6156839  ... -1.74750652 -3.01758057
  2.44781043]
[ 3.42443605  1.47461764  2.67335101 ... -0.26221732 -2.47159285
  1.07550703]
[ 2.90137293  0.41838916  1.05044245 ... -2.73824467 -2.12413492
  0.93109852]
[ 4.0413531   2.28634022  1.33837476 ... -0.52447036 -2.33999363
  2.22113846]
[ 2.9851779   2.31855532  2.6747473  ... -0.31690819 -2.25866203
  1.85797757]
[ 3.13770501  0.03460418  0.54079779 ... -2.03949606 -2.81616086
  0.94021924]
[ 2.82869561  1.8713737   0.83154584 ... -0.34584707 -1.37997277
  1.21587619]
[ 3.44128874  1.65715724  2.327772   ... -0.11882372 -1.29861905
  1.63594538]
[ 3.70264089  1.86356057  1.98898149 ... -1.51413652 -2.97764472
  1.93133243]
[ 1.42725609  1.11414264  0.73235963 ... -1.58271949 -1.79962872
  1.76971557]
[ 2.81888962  1.10255157  2.58213399 ...  0.37139763 -0.79758638
  1.09498972]
[ 2.34283556  0.07830037  0.97731049 ... -0.08342334

[ 3.09367111  1.50809134  1.40917694 ... -0.83204944 -2.33596305
  1.89983201]
[ 2.98665018  1.32946424  1.42274575 ... -1.66167583 -2.41434377
  0.96478419]
[ 1.7771414   0.16196777  0.71707378 ...  0.28186503 -0.92495957
  0.51083525]
[ 2.90485927  3.4997186   0.70118198 ... -1.09848244 -0.1975819
  2.55115767]
[ 3.50552558  1.78988996  1.45516944 ... -1.86275037 -1.56742618
  1.66685027]
[ 3.46198937  1.23938381  0.15141346 ... -1.42759432 -0.68336183
  0.32522105]
[ 2.11105661  2.25557329  1.12516454 ... -1.53802572 -0.67914331
  2.79998949]
[ 3.16549532  0.19977848  0.234073   ... -0.60775519 -2.4529423
  0.26150747]
[ 2.15277349  0.37251204  0.68109299 ... -1.22629668 -0.28110631
  1.4542529 ]
[ 3.21408216  1.01537836  1.85389292 ...  0.12751946 -3.5095828
  1.68043008]
[ 3.3399683   0.86267002  1.18558548 ... -1.09856923 -2.2742975
  0.7605473 ]
[ 2.79792359  1.49894196  1.99297441 ... -1.02766885 -3.36871038
  2.30906814]
[ 0.82376622  1.5444563  -0.91250586 ... -3.62555565 -0.

[ 4.50148234  2.31221629  2.25326558 ... -1.15585881 -3.37050194
  2.13688341]
[ 1.94852278  0.01368169  0.50362888 ... -2.06889885 -1.24137171
  0.89901665]
[ 3.86328895  0.41550087  1.18887083 ... -1.7132871  -2.45177571
  1.48947182]
[ 3.34644275  1.61426734  2.07207332 ... -1.14762365 -3.55437242
  3.03998029]
[ 2.99599361  1.52027731  1.6503721  ... -1.42254506 -3.91088311
  2.15864378]
[ 2.72569547  1.43046841  1.6757933  ... -0.52779121 -2.32967962
  1.76853183]
[ 2.81017542  0.99214432  1.46117345 ... -1.74744233 -2.579536
  2.43345396]
[ 3.95262453  0.41395427  0.93409277 ... -1.83774757 -1.33306921
  1.15979001]
[ 2.97466992  2.12922731  1.03578201 ... -1.55626427 -3.66278296
  1.23825244]
[ 3.49708336  2.00165062  2.72422957 ...  0.21578224 -3.70549894
  3.19071883]
[ 3.13233904  0.01823319  1.16664699 ... -0.24149777 -1.71843511
  0.65285415]
[ 4.48020599  1.64721331  2.42591804 ... -0.35992134 -3.10870124
  3.16771889]
[ 3.19940163  1.56044915  3.01589255 ...  1.1718025  -

[ 1.74088784  0.93295966  1.45578283 ... -0.99221792 -2.33940935
  1.92847019]
[ 3.95934295 -0.1013416   1.20010066 ... -0.85697872 -0.03243704
  2.45338869]
[ 4.25062931  1.31478791  2.07187982 ... -0.59535221 -3.51668497
  2.12081691]
[ 1.36977106  0.8998534   1.30175894 ...  1.12236956 -0.104854
 -0.55094591]
[ 2.34516712  0.24653422  0.50002894 ... -1.26131658 -0.1397795
  0.91967149]
[ 4.27873055  0.70012867  1.83657095 ...  0.91106234 -1.67833422
  2.20210977]
[ 3.40708104  1.01939015  1.40992231 ... -2.60874447 -3.42999446
  1.74266348]
[ 3.55538653  2.19767092  0.68590471 ... -1.44289764 -1.00806273
  2.32479653]
[ 2.42908527  1.44074934 -0.01711391 ... -3.15540572 -0.68066032
  1.59991794]
[ 3.45226271  1.97710998  1.99741256 ... -0.07739216 -1.78359112
  0.98417681]
[ 2.35261589  0.97136248  1.67525298 ... -0.81281323 -1.84882912
  1.37607812]
[ 3.81313769  0.61549053  1.36602074 ... -0.98881666 -2.05918887
  1.17132288]
[ 3.38614964  0.77956618  1.3563565  ... -1.42005725 -1

[ 3.86535823  2.76619717  0.71014375 ... -1.78739501 -2.13517576
  2.28897136]
[ 2.10167711  1.16921179  1.14433472 ... -0.35620635 -1.02103514
  1.21021499]
[ 2.35122874  1.48093093  1.39376692 ... -0.44141409 -1.37889016
  2.41060397]
[ 3.08304936  1.0302005   1.32437467 ... -0.84323971 -2.83047446
  2.99800149]
[3.02177875 1.89627966 0.83376633 ... 0.17207515 0.07656984 3.38131272]
[ 3.97831748  1.97347656  2.882758   ... -2.08450859 -5.3184312
  1.44498937]
[ 3.52562438  0.11147238  1.75499338 ...  0.34606742 -0.47201372
  1.32101655]
[ 3.93255092  1.46071705  2.54587657 ...  0.41627967 -2.97175993
  3.51497556]
[ 3.58261446  2.63177032  2.4694519  ... -0.61133798 -3.3889618
  2.39660333]
[ 3.09099635  1.26902064  1.18409234 ... -1.27015605 -1.69625737
  2.62922185]
[ 2.39236036  0.98746503  0.77902809 ... -1.55229905 -1.7721429
  1.50979544]
[ 3.31219272  1.39091522  1.48171295 ... -1.24299458 -2.15870051
  3.07059676]
[ 3.71092332  2.0503946   2.36477937 ... -1.0352561  -3.485504

In [13]:
bpr.get_similar_items(N = 5)

array([[ 309, 1094, 3186, 2166, 2664],
       [ 639,  537, 1439, 2956,   68],
       [3032, 1536, 2099, 1826, 3248],
       ...,
       [1480, 3201, 1488, 1313, 1086],
       [3414, 1692, 3597, 3152, 1411],
       [1241, 2469, 3617, 2497, 2748]], dtype=uint32)

In [14]:
bpr.recommend(X_train, N = 5)

scores
[ 3.92037176  3.27680457  1.26365154 ... -0.55006471 -3.05304743
  3.46472965]
scores
[ 3.56094197  1.17763847  1.93899515 ... -1.59849528 -2.69070004
  1.95123894]
scores
[ 3.31529125  1.21372552  1.70971963 ... -0.44382261 -1.30192284
  2.08701166]
scores
[ 3.62201974  2.26395876  2.50396302 ... -1.72524495 -3.6536141
  1.99534518]
scores
[ 2.37084626 -0.14559474  0.4366446  ... -0.60603417 -0.27899496
  0.95526757]
scores
[ 2.5545919   2.51186329  1.34747792 ... -2.11453457 -0.89110331
  0.93001346]
scores
[ 3.7794387   2.6478506   2.28642557 ... -2.03509083 -3.6479362
  3.05281082]
scores
[ 2.39842854  1.10360123  2.02969607 ... -0.34077662 -2.16406893
  0.4882391 ]
scores
[ 4.15367015  1.27425784  2.17514011 ... -0.72753535 -2.40752157
  1.68326371]
scores
[ 2.97603661  2.48456889  0.92867898 ... -1.50735769 -1.98603572
  2.81936489]
scores
[ 2.81145929  0.16173266  1.71947563 ... -2.0191673  -1.9022126
  2.19619489]
scores
[ 3.59986158  1.40304015 -0.13115077 ... -2.210387

scores
[ 3.61693524  0.50826865  0.40912209 ... -3.24913794 -1.20674138
  1.22946322]
scores
[ 3.02417982  2.61120409  2.29833111 ... -1.02944098 -3.27718352
  1.47945803]
scores
[ 2.62440411  0.91612872  1.19505295 ... -1.21333713 -1.45363837
  1.22262615]
scores
[ 2.26706703  1.29610244  1.74352272 ... -0.48983678 -2.0464789
  1.17943981]
scores
[ 3.4179627   1.92088691  1.41520086 ... -1.48627846 -2.50004927
  1.12821373]
scores
[ 2.64733704  1.48361215  2.46679582 ... -0.66489075 -3.05759804
  1.63906396]
scores
[ 3.21300389  0.82952152  1.49050791 ... -0.49630851 -1.69616115
  1.60017938]
scores
[ 2.27789917  0.86785029  0.06024573 ... -1.65045774 -1.15692331
  1.86403935]
scores
[ 3.52554763  1.33616275  2.0269547  ... -0.68966786 -3.54986875
  2.34521031]
scores
[ 2.47488368  2.60553954  1.70485116 ... -0.90538872 -1.33687054
  2.26315448]
scores
[ 3.43962907  1.37951104  1.6232994  ... -1.85787359 -2.60254934
  1.95252684]
scores
[ 3.21682913  1.78367786  2.58573877 ...  1.0246

scores
[ 4.42933223  0.90239956  1.76307037 ... -1.31009377 -1.89966172
  1.98233904]
scores
[ 3.91285429  1.36414544  2.13313725 ... -0.39498522 -3.82614314
  2.27496576]
scores
[ 4.72196177  1.11815632  2.44759523 ... -1.34799955 -2.68119392
  2.27657827]
scores
[ 3.07896205  0.70679896  1.68541045 ... -1.38171725 -1.46786272
  0.41509186]
scores
[1.74837915 0.69265511 1.06464225 ... 0.10573886 0.70099746 0.3949218 ]
scores
[ 2.02704322  0.61961487  1.33434567 ... -1.12765823 -0.31629304
  3.28666359]
scores
[ 0.85343447 -0.06920628  1.21481158 ... -2.40680707 -4.38459961
 -0.28370749]
scores
[ 3.46096569  2.3598683   1.45409656 ... -2.88487872 -2.60490376
  1.88497539]
scores
[ 2.56868123  1.75093039  1.7358889  ... -0.2755245  -1.38405684
  1.87213813]
scores
[ 2.36420415  2.02539951  2.3012274  ... -0.24740638 -1.77122212
  2.78145382]
scores
[ 3.04386297  0.12732459  1.10163381 ... -2.2267181  -1.36776126
  2.34616679]
scores
[ 2.76934738  0.6190841   1.31844    ... -1.46345013 -

scores
[ 4.16128374  3.58990093  1.01829011 ... -2.52332447 -0.69958509
  3.20860517]
scores
[ 3.2227967   2.057158    1.52554323 ... -0.45824903 -1.54347372
  2.18159187]
scores
[ 3.17680051  0.94384324  0.93878274 ...  0.02250931 -0.82028879
  0.64082711]
scores
[ 3.39251658  2.0234424   2.20931031 ... -2.06653346 -3.64065822
  3.49505078]
scores
[ 1.71339836  0.18914577  0.39783859 ...  0.76931212 -0.65388337
  1.35013444]
scores
[ 2.20991117  1.19415623  1.95980478 ... -0.86101522 -1.56215939
  2.16785184]
scores
[ 1.56358748  0.68955953  1.93728411 ... -0.32936394 -1.58535522
  1.222517  ]
scores
[ 2.94810885 -0.10628981  1.11471956 ... -0.18308641 -0.83901679
  0.80583319]
scores
[ 3.62475611  0.67151278  1.17792656 ... -0.72626877 -1.86233982
  2.76888913]
scores
[ 2.9907752   0.40829333  1.47097846 ... -0.78084665 -1.55266273
  2.53340319]
scores
[ 2.71732275  1.6184106   3.36605518 ...  1.86057903 -1.22427147
  2.03201279]
scores
[ 3.31339019  0.91812123  1.90680407 ...  0.538

scores
[ 3.23679396  3.23050925  2.25103716 ... -1.02570143 -2.83446136
  2.62482705]
scores
[ 2.93854856  0.71750102  2.28275865 ... -2.25550489 -2.97416224
  2.00662875]
scores
[ 2.88624153  1.41382585  1.89255326 ... -1.01503587 -2.36353115
  2.21590449]
scores
[ 3.34704376  1.28991074  2.96313976 ... -0.74871751 -2.67816464
  3.75851354]
scores
[ 2.59541574  2.8524613   2.10907398 ...  0.48366215 -2.22148999
  3.03377191]
scores
[ 4.74145512  1.96133504  0.14480924 ... -4.07570741 -2.82307713
  1.447255  ]
scores
[ 3.33505812  2.07388048  2.37734325 ...  0.19160616 -0.8872043
  2.59617162]
scores
[ 4.92334571  1.65798716  3.38550497 ...  1.10358669 -2.70025384
  1.83423481]
scores
[ 3.64520723  1.74983449  2.51534738 ... -0.76875439 -3.25772286
  2.79169797]
scores
[ 3.26543786  1.86745932  2.19703543 ... -1.14063704 -3.38055513
  2.5294488 ]
scores
[ 4.46009208  0.02023327  2.31315951 ...  0.58952091 -1.38782198
  1.81210806]
scores
[ 2.33562593  0.50125299  0.53813232 ... -0.6850

...

scores
[ 2.76023849 -0.16795636  0.09274821 ... -2.30055073 -1.43192829
  0.80478218]
scores
[ 2.58970794  0.69529627  0.91020313 ... -1.84108813 -1.7750392
  1.27504096]
scores
[ 3.37073598 -0.45623501  1.00806662 ... -0.54315455 -2.20464479
  1.76982214]
scores
[ 3.7401555   1.27805697  1.8829294  ... -1.48740329 -3.31933378
  2.00112747]
scores
[ 2.18047039  0.77154854  0.71779231 ... -1.59474592 -1.37164627
  2.00799157]
scores
[ 2.39476993  0.70186001  0.95447648 ...  0.10556175 -1.11890628
  2.18166157]
scores
[ 3.81048283  1.32328922  2.06703435 ... -1.08510572 -1.47935881
  2.14209927]
scores
[ 3.58385831  1.47339338  2.56442744 ... -1.77157554 -4.34371904
  2.30645117]
scores
[ 2.04901468  1.21938343  1.45389926 ... -1.79082991 -1.38428453
  2.06946207]
scores
[ 1.9904137  -0.3606828  -0.09106415 ... -1.37495219  0.88704799
  0.44136698]
scores
[ 4.20523294  0.42587021  0.69017741 ... -1.73782209 -2.33118422
  0.91558778]
scores
[ 4.39628788  1.11472008  0.93001556 ... -1.2114

scores
[ 2.87569702  1.6390683   3.65573709 ...  1.53169171 -0.27622435
  1.13635629]
scores
[ 2.89416903  0.89419151 -0.0121479  ... -2.16532716 -1.21926323
  2.17376506]
scores
[ 4.68273333  0.55831878  1.11428121 ... -2.11855021 -3.95370429
  2.72973624]
scores
[ 2.41955718  0.88430096  0.99669992 ... -2.14853759 -1.25009004
  0.87457725]
scores
[ 3.76374429  0.98079633  2.35592378 ... -0.72603797 -3.13417601
  2.21510267]
scores
[ 3.60800604  1.96490636  3.30607752 ...  0.68355847 -2.67408284
  2.29887683]
scores
[ 3.91585233  1.43029418  0.59772326 ... -2.22233396 -1.61431575
  1.37263749]
scores
[ 3.77085841  2.09761168  1.90286484 ... -0.77656987 -2.84995772
  1.51781034]
scores
[ 3.05508374  0.21543478  0.53101289 ... -2.66326889 -2.43297852
  2.56903504]
scores
[ 1.9573587   1.36356184  0.25637605 ... -0.53486016  0.93242144
  1.34873588]
scores
[ 3.21436975  0.10226148  1.15387642 ... -1.7740534  -2.40151069
  0.77312097]
scores
[ 4.03729409  0.19569682  2.36897894 ... -0.493

scores
[ 2.44668037  2.02898681  0.53181108 ... -2.34493169 -3.35035342
  1.99378149]
scores
[ 3.76251152  2.03406266  1.78068956 ... -2.78247497 -2.94280002
  3.13768613]
scores
[ 2.91600866  2.22158626  1.71627497 ... -1.57688462 -3.60987586
  1.5094674 ]
scores
[ 4.22644551  1.53865948  2.31667547 ...  0.3394258  -2.95708922
  2.92777139]
scores
[ 3.83001443  1.12858964  0.85635582 ... -2.54107489 -2.39794772
  1.66967809]
scores
[ 1.74088784  0.93295966  1.45578283 ... -0.99221792 -2.33940935
  1.92847019]
scores
[ 3.95934295 -0.1013416   1.20010066 ... -0.85697872 -0.03243704
  2.45338869]
scores
[ 4.25062931  1.31478791  2.07187982 ... -0.59535221 -3.51668497
  2.12081691]
scores
[ 1.36977106  0.8998534   1.30175894 ...  1.12236956 -0.104854
 -0.55094591]
scores
[ 2.34516712  0.24653422  0.50002894 ... -1.26131658 -0.1397795
  0.91967149]
scores
[ 4.27873055  0.70012867  1.83657095 ...  0.91106234 -1.67833422
  2.20210977]
scores
[ 3.40708104  1.01939015  1.40992231 ... -2.608744

scores
[ 4.78596007  1.55717969  1.89989933 ... -0.11893724 -2.07503529
  2.31512687]
scores
[ 2.79023444  0.77146848  1.44454561 ... -0.86200016 -1.06707302
  1.73612504]
scores
[ 3.43249102  1.39125213  1.60906998 ... -1.85206814 -2.40503519
  1.28988159]
scores
[ 3.50353312  0.94321297  0.84032356 ... -1.86412883 -0.55098443
  1.52890633]
scores
[ 3.81682906  0.40714867  1.65891787 ... -0.64588854 -1.85019147
  0.75611207]
scores
[ 2.15174644 -0.51170353  1.15192793 ...  0.2494023  -0.81464212
  0.90240875]
scores
[ 2.36452358  0.31896274  1.58166052 ... -1.44317909 -3.66354273
  1.25950455]
scores
[ 4.37766837 -0.13165632  0.20213372 ... -2.49284861 -0.35555254
  2.36996986]
scores
[ 3.86535823  2.76619717  0.71014375 ... -1.78739501 -2.13517576
  2.28897136]
scores
[ 2.10167711  1.16921179  1.14433472 ... -0.35620635 -1.02103514
  1.21021499]
scores
[ 2.35122874  1.48093093  1.39376692 ... -0.44141409 -1.37889016
  2.41060397]
scores
[ 3.08304936  1.0302005   1.32437467 ... -0.843

array([[1173,  851,  859, 1186,  309],
       [1025,  593, 2426,  253,  851],
       [2400,  579, 2958, 2494, 2480],
       ...,
       [1131, 1124,  253,  309, 1123],
       [1215,    0, 1124,  309,  346],
       [1117, 2898, 1113,  713,  847]], dtype=uint32)